##### データセットの場所やバッチサイズなどの定数値の設定

In [1]:
import os

# 使用するデバイス
DEVICE = 'cpu'

# エポック数
N_EPOCHS = 20

# 学習時のバッチサイズ
BATCH_SIZE = 100

DATASET_CSV = './fruits-360/train_list.csv'

# 画像ファイルの先頭に付加する文字列
DATA_DIR  = './fruits-360/'

# 画像サイズ
H = 100 # 縦幅
W = 100 # 横幅
C = 3 # チャンネル数（カラー画像なら3，グレースケール画像なら1）

# 特徴ベクトルの次元数
N = 128

# 学習結果の保存先フォルダ
MODEL_DIR = './AE_models_Banana/'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE_ENC = os.path.join(MODEL_DIR, 'banana_encoder_model.pth') # エンコーダ
MODEL_FILE_DEC = os.path.join(MODEL_DIR, 'banana_decoder_model.pth') # デコーダ


##### ニューラルネットワークモデルの定義

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# バナナの画像をN次元特徴ベクトルへと圧縮するニューラルネットワーク
class BananaEncoder(nn.Module):
    
    # C: 入力顔画像のチャンネル数（1または3と仮定）
    # H: 入力顔画像の縦幅（8の倍数と仮定）
    # W: 入力顔画像の横幅（8の倍数と仮定）
    # N: 出力の特徴ベクトルの次元数
    def __init__(self, C, H, W, N):
        super(BananaEncoder, self).__init__()
        
        # カーネルサイズ（）
        self.conv1 = nn.Conv2d(in_channels=C, out_channels=16, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2, padding=1)
        
        # バッチ正規化層
        self.bn1 = nn.BatchNorm2d(num_features=16)
        self.bn2 = nn.BatchNorm2d(num_features=32)
        self.bn3 = nn.BatchNorm2d(num_features=64)
        
        # 平坦化
        self.flat = nn.Flatten()
        
        # 全結合層1
        self.fc1 = nn.Linear(in_features=H*W, out_features=2048)
        
        # 全結合層2
        self.fc2 = nn.Linear(in_features=2048, out_features=N)
        
    def forward(self, x):
        h = F.leaky_relu(self.bn1(self.conv1(x)))
        h = F.leaky_relu(self.bn2(self.conv2(h)))
        h = F.leaky_relu(self.bn3(self.conv3(h)))
        # h = F.leaky_relu(self.bn4(self.conv4(h)))
        #h = self.conv4(h) # 例: Residual Block を使用する場合はこのように記載（Residual Blockの内部でバッチ正規化と活性化関数を適用しているので，外側では適用しない）
        h = self.flat(h)
        h = F.leaky_relu(self.fc1(h))
        z = self.fc2(h)
        return z
        
# N 次元の特徴ベクトルから顔画像を生成するニューラルネットワーク
# AutoEncoderのデコーダ部分のサンプル
class BananaDecoder(nn.Module):

    # C: 出力顔画像のチャンネル数（1または3と仮定）
    # H: 出力顔画像の縦幅（8の倍数と仮定）
    # W: 出力顔画像の横幅（8の倍数と仮定）
    # N: 入力の特徴ベクトルの次元数
    def __init__(self, C, H, W, N):
        super(BananaDecoder, self).__init__()
        self.W = W
        self.H = H

        # 全結合層1,2
        # パーセプトロン数は FaceEncoder の全結合層と真逆に設定
        self.fc2 = nn.Linear(in_features=N, out_features=2048)
        self.fc1 = nn.Linear(in_features=2048, out_features=H*W)

        # 逆畳込み層1～4
        # カーネルサイズ，ストライド幅，パディングは FaceEncoder の畳込み層1～4と真逆に設定
        # self.deconv4 = nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.deconv3 = nn.ConvTranspose2d(in_channels=64, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose2d(in_channels=32, out_channels=16, kernel_size=4, stride=2, padding=1)
        self.deconv1 = nn.ConvTranspose2d(in_channels=16, out_channels=8, kernel_size=4, stride=2, padding=1)

        # バッチ正規化層
        # self.bn4 = nn.BatchNorm2d(num_features=64)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.bn2 = nn.BatchNorm2d(num_features=16)
        self.bn1 = nn.BatchNorm2d(num_features=8)

        # 畳込み層
        # 逆畳込み層の出力には checker board artifact というノイズが乗りやすいので，最後に畳込み層を通しておく
        self.conv = nn.Conv2d(in_channels=8, out_channels=C, kernel_size=3, stride=1, padding=1)

    def forward(self, z):
        h = F.leaky_relu(self.fc2(z))
        h = F.leaky_relu(self.fc1(h))
        h = torch.reshape(h, (len(h), 64, self.H//8, self.W//8)) # 一列に並んだパーセプトロンを 64*(H/8)*(W/8) の特徴マップに並べ直す
        # h = F.leaky_relu(self.bn4(self.deconv4(h)))
        h = F.leaky_relu(self.bn3(self.deconv3(h)))
        h = F.leaky_relu(self.bn2(self.deconv2(h)))
        h = F.leaky_relu(self.bn1(self.deconv1(h)))
        y = torch.sigmoid(self.conv(h))
        return y

##### 訓練データセットの読み込み

In [3]:
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from mylib.data_io import CSVBasedDataset
from mylib.utility import save_datasets, load_datasets_from_file


# 前回の試行の続きを行いたい場合は True にする -> 再開モードになる
RESTART_MODE = False


# 再開モードの場合は，前回使用したデータセットをロードして使用する
if RESTART_MODE:
    train_dataset, valid_dataset = load_datasets_from_file(MODEL_DIR)
    if train_dataset is None:
        print('error: there is no checkpoint previously saved.')
        exit()
    train_size = len(train_dataset)
    valid_size = len(valid_dataset)

# そうでない場合は，データセットを読み込む
else:

    # CSVファイルを読み込み, 訓練データセットを用意
    dataset = CSVBasedDataset(
        filename = DATASET_CSV,
        items = [
            'File Path', # X
        ],
        dtypes = [
            'image', # Xの型
        ],
        dirname = DATA_DIR,
        img_transform = transforms.CenterCrop((H, W)), # 処理量を少しでも抑えるため，画像中央の H×W ピクセルの部分だけを対象とする
    )  
    
    # 訓練データセットを分割し，一方を検証用に回す
    dataset_size = len(dataset)
    valid_size = int(0.002 * dataset_size) # 全体の 0.2% を検証用に -> tinyCelebA の画像は全部で 16000 枚なので，検証用画像は 16000*0.002=32 枚
    train_size = dataset_size - valid_size # 残りの 99.8% を学習用に
    train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

    # データセット情報をファイルに保存
    save_datasets(MODEL_DIR, train_dataset, valid_dataset)

# 訓練データおよび検証用データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

c:\Users\HIROYUKI\anaconda3\envs\envA\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 指定されたプロシージャが見つかりません。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


##### 学習処理の実行

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from mylib.visualizers import LossVisualizer
from mylib.data_io import show_images
from mylib.utility import save_checkpoint, load_checkpoint


# 前回の試行の続きを行いたい場合は True にする -> 再開モードになる
RESTART_MODE = False


# エポック番号
INIT_EPOCH = 0 # 初期値
LAST_EPOCH = INIT_EPOCH + N_EPOCHS # 最終値

# ニューラルネットワークの作成
enc_model = BananaEncoder(C=C, H=H, W=W, N=N).to(DEVICE)
dec_model = BananaDecoder(C=C, H=H, W=W, N=N).to(DEVICE)

# 最適化アルゴリズムの指定（ここでは SGD でなく Adam を使用）
enc_optimizer = optim.Adam(enc_model.parameters())
dec_optimizer = optim.Adam(dec_model.parameters())

# 再開モードの場合は，前回チェックポイントから情報をロードして学習再開
# if RESTART_MODE:
#     INIT_EPOCH, LAST_EPOCH, enc_model, enc_optimizer = load_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_ENC_MODEL, CHECKPOINT_ENC_OPT, N_EPOCHS, enc_model, enc_optimizer)
#     _, _, dec_model, dec_optimizer = load_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_DEC_MODEL, CHECKPOINT_DEC_OPT, N_EPOCHS, dec_model, dec_optimizer)
#     print('')

# 損失関数
loss_func = nn.MSELoss()

# 損失関数値を記録する準備
loss_viz = LossVisualizer(['train loss', 'valid loss'], init_epoch=INIT_EPOCH)

# 勾配降下法による繰り返し学習
for epoch in range(INIT_EPOCH, LAST_EPOCH):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    enc_model.train()
    dec_model.train()
    sum_loss = 0
    # for X in tqdm(train_dataloader):
    for X in train_dataloader:
        for param in enc_model.parameters():
            param.grad = None
        for param in dec_model.parameters():
            param.grad = None
        X = X.to(DEVICE)
        Z = enc_model(X) # 入力画像 X を現在のエンコーダに入力し，特徴ベクトル Z を得る
        Y = dec_model(Z) # 特徴ベクトル Z を現在のデコーダに入力し，復元画像 Y を得る
        loss = loss_func(X, Y) # 損失関数の現在値を計算
        loss.backward() # 誤差逆伝播法により，個々のパラメータに関する損失関数の勾配（偏微分）を計算
        enc_optimizer.step() # 勾配に沿ってパラメータの値を更新
        dec_optimizer.step() # 同上
        sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / train_size
    loss_viz.add_value('train loss', avg_loss) # 訓練データに対する損失関数の値を記録
    print('train loss = {0:.6f}'.format(avg_loss))

    # 検証
    enc_model.eval()
    dec_model.eval()
    sum_loss = 0
    with torch.inference_mode():
        # for X in tqdm(valid_dataloader):
        for X in valid_dataloader:
            X = X.to(DEVICE)
            Z = enc_model(X)
            Y = dec_model(Z) 
            loss = loss_func(X, Y)
            sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / valid_size
    loss_viz.add_value('valid loss', avg_loss) # 検証用データに対する損失関数の値を記録
    print('valid loss = {0:.6f}'.format(avg_loss))
    print('')

    # 学習経過の表示
    if epoch == 0:
        show_images(X.to('cpu').detach(), num=BATCH_SIZE, num_per_row=8, title='original', save_fig=False, save_dir=MODEL_DIR)
    show_images(Y.to('cpu').detach(), num=BATCH_SIZE, num_per_row=8, title='epoch {0}'.format(epoch + 1), save_fig=False, save_dir=MODEL_DIR)

    # # 現在の学習状態を一時ファイル（チェックポイント）に保存
    # save_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_ENC_MODEL, CHECKPOINT_ENC_OPT, epoch+1, enc_model, enc_optimizer)
    # save_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_DEC_MODEL, CHECKPOINT_DEC_OPT, epoch+1, dec_model, dec_optimizer)

# 学習結果のニューラルネットワークモデルをファイルに保存
enc_model = enc_model.to('cpu')
dec_model = dec_model.to('cpu')
torch.save(enc_model.state_dict(), MODEL_FILE_ENC)
torch.save(dec_model.state_dict(), MODEL_FILE_DEC)

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

Epoch 1:


AttributeError: '_OpNamespace' 'image' object has no attribute 'read_file'